In [1]:
!. ./path.sh

In [ ]:
import argparse
import numpy as np
import wave
import sys
import time
sys.path.append('../..')
#import espnet
#from espnet_model_zoo.downloader import ModelDownloader
#from espnet2.bin.asr_inference_streaming import Speech2TextStreaming
from espnet2.bin.asr_transducer_inference import Speech2Text
#from espnet2.bin.asr_inference_dual_delay_streaming import Speech2TextStreaming

In [ ]:
speech2text = Speech2Text(
    #asr_base_path="/mnt/aoni04/jsakuma/development/espnet-g05-1.8/egs2/atr3/asr1",
    asr_train_config="/mnt/aoni04/jsakuma/development/espnet-g05-1.8/egs2/csj_core/asr1/exp/asr_train_asr_cbs_transducer_848_raw_jp_char_sp/config.yaml",
    asr_model_file="/mnt/aoni04/jsakuma/development/espnet-g05-1.8/egs2/csj_core/asr1/exp/asr_train_asr_cbs_transducer_848_raw_jp_char_sp/valid.loss_transducer.ave_10best.pth",
#     lm_train_config="/mnt/aoni04/jsakuma/development/espnet-g05-1.8/egs2/mycsj2/asr1/exp/lm_train_lm_jp_char/config.yaml",
#     lm_file="/mnt/aoni04/jsakuma/development/espnet-g05-1.8/egs2/mycsj2/asr1/exp/lm_train_lm_jp_char/valid.loss.best.pth",
    token_type=None,
    bpemodel=None,
#     maxlenratio=0.0,
#     minlenratio=0.0,
    beam_size=10,
#     ctc_weight=0.5,
    lm_weight=0.0,
#     penalty=0.0,
    nbest=1,
    device = "cpu",
    streaming = True,
#     disable_repetition_detection=True,
#     decoder_text_length_limit=0,
#     encoded_feat_length_limit=0
)

In [ ]:
import wave
import struct

def read_wav(wavpath):
    wf = wave.open(wavpath, 'r')

    # waveファイルが持つ性質を取得
    ch = wf.getnchannels()
    width = wf.getsampwidth()
    fr = wf.getframerate()
    fn = wf.getnframes()

    x = wf.readframes(wf.getnframes()) #frameの読み込み
    x = np.frombuffer(x, dtype= "int16") #numpy.arrayに変換

    return x

In [ ]:
#wavfile = "/mnt/aoni04/jsakuma/data/ATR-Fujie/samples/wav/20131101-3_01/20131101-3_01_001_ch0.wav"
wavfile = "../../mycsj2/asr1/dump/raw/eval1/data/format.1/A01M0097_0000211_0026743.flac"

In [ ]:
# wav = read_wav(wavfile)

In [ ]:
# wavfile = "dump/raw/eval1/data/format.1/A01M0097_0000211_0026743.flac"

In [ ]:
import soundfile as sf
wav, samplerate = sf.read(wavfile)

In [ ]:
wav.shape, type(wav), samplerate

In [ ]:
import IPython
rate = 16000
IPython.display.Audio(wav, rate=rate)

In [ ]:
len(wav) / 800, len(wav), len(wav) / 16000

In [ ]:
def get_recognize_frame(data):
    output_list = []
    speech = data#.astype(np.float16)/32767.0 #32767 is the upper limit of 16-bit binary numbers and is used for the normalization of int to float.
    sim_chunk_length =  800 # 640

    if sim_chunk_length > 0:
        for i in range(len(speech)//sim_chunk_length):
            results = speech2text.streaming_decode(speech=speech[i*sim_chunk_length:(i+1)*sim_chunk_length], is_final=False)
            #print(results)
            if results is not None and len(results) > 0:
                nbests = [text for text, token, token_int, hyp in results]
                text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
                #progress_output(nbests[0])
                output_list.append(text)
            else:
                #progress_output("")
                output_list.append("")

        results = speech2text.streaming_decode(speech[(i+1)*sim_chunk_length:len(speech)], is_final=True)
    else:
        results = speech2text.streaming_decode(speech, is_final=True)
    
    nbests = [text for text, token, token_int, hyp in results]
    if results is not None and len(results) > 0:
        nbests = [text for text, token, token_int, hyp in results]
        text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
        #progress_output(nbests[0])
        output_list.append(text)
    else:
        #progress_output("")
        output_list.append("")
    return output_list
    #progress_output(nbests[0])

In [ ]:
start = time.perf_counter()
text = get_recognize_frame(wav)
end = time.perf_counter()
print(end-start)

In [14]:
text[-1]

'えー内容としましては生成過程の戻るモデルに基づく付るパターンの分析各音節の分節的特徴とえー音調しやとの間の相体的な時間関係の記憶がえ音長試練の大きさのヒノキ化えー音調し例とフレーズレの大きさの両しか帰則によって生成されたエフェロパターンを持つ合成音声ので妄想の順でお話ししたいと思います'

In [15]:
len(text)

531

In [95]:
start = time.perf_counter()
text = get_recognize_frame(wav)
end = time.perf_counter()
print(end-start)

26.57570191565901


In [96]:
text[-1]

'あのすいませんフランス料理屋を都内で探しているんですけどーえっとーそうだなちょっと家族の誕生日ように使うんですけどーえーとー日暮里の金近くでは何かーいいお店ありますかね。'

In [75]:
# text2, results_list = get_recognize_frame2(wav)

In [76]:
len(text)

259

In [77]:
text[-1]

'あのすいませんフランス料理屋を都内で探しているんですけどーえっとーそうだなちょっと家族の誕生日ように使うんですけどーえーとー日暮里の金近くでは何かーいいお店ありますかね。'

In [32]:
text[-1]

'あのすいませんフランス料理屋を都内で探しているんですけどーえっとーそうだなちょっと家族の誕生日用に使うんですけどえーとーそういうのちょってあのあのちょっと家族のたいいいでもあるんですかね。'

In [17]:
text[-1]

'あのすいませんフランス料理屋を都内で探しているんですけどーえっとーそうだなちょと家族の誕生日用に使うんですけどーえーっとーん日本に近くできなかがいい店ありますかね。'

In [37]:
800 / 16000

0.05

In [32]:
text[1000]

'あのすいませんフランス料理屋を都内で探しているんですけどえっとーそうだなちょと家族の誕生日用に使うん'

In [22]:
wav.shape

(206400,)

In [99]:
prev_lines = 0
def progress_output(text):
    global prev_lines
    lines=['']
    for i in text:
        if len(lines[-1]) > 100:
            lines.append('')
        lines[-1] += i
    for i,line in enumerate(lines):
        if i == prev_lines:
            #print('\n\r')
            sys.stderr.write('\n\r')
            #pass
        else:
            #print('\r\033[B\033[K')
            sys.stderr.write('\r\033[B\033[K')
            #pass
            
        #print(line)
        sys.stderr.write(line)

    prev_lines = len(lines)
    sys.stderr.flush()

In [100]:
def recognize_from_file(wavfile):
    with wave.open(wavfile, 'rb') as wavfile:
        ch=wavfile.getnchannels()
        bits=wavfile.getsampwidth()
        rate=wavfile.getframerate()
        nframes=wavfile.getnframes()
        buf = wavfile.readframes(-1)
        data=np.frombuffer(buf, dtype='int16')
    speech = data.astype(np.float16)/32767.0 #32767 is the upper limit of 16-bit binary numbers and is used for the normalization of int to float.
    sim_chunk_length = 800 # 640
    print(len(speech))
    if sim_chunk_length > 0:
        for i in range(len(speech)//sim_chunk_length):
            results = speech2text(speech=speech[i*sim_chunk_length:(i+1)*sim_chunk_length], is_final=False)
            if results is not None and len(results) > 0:
                nbests = [text for text, token, token_int, hyp in results]
                text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
                progress_output(nbests[0])
            else:
                progress_output("")

        results = speech2text(speech[(i+1)*sim_chunk_length:len(speech)], is_final=True)
    else:
        results = speech2text(speech, is_final=True)
    nbests = [text for text, token, token_int, hyp in results]
    progress_output(nbests[0])
    
def recognize(data):
#     with wave.open(wavfile, 'rb') as wavfile:
#         ch=wavfile.getnchannels()
#         bits=wavfile.getsampwidth()
#         rate=wavfile.getframerate()
#         nframes=wavfile.getnframes()
#         buf = wavfile.readframes(-1)
#         data=np.frombuffer(buf, dtype='int16')
        
    speech = data.astype(np.float16)/32767.0 #32767 is the upper limit of 16-bit binary numbers and is used for the normalization of int to float.
    sim_chunk_length = 800 # 640
    print(len(speech))
    if sim_chunk_length > 0:
        for i in range(len(speech)//sim_chunk_length):
            results = speech2text(speech=speech[i*sim_chunk_length:(i+1)*sim_chunk_length], is_final=False)
            if results is not None and len(results) > 0:
                nbests = [text for text, token, token_int, hyp in results]
                text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
                progress_output(nbests[0])
            else:
                progress_output("")

        results = speech2text(speech[(i+1)*sim_chunk_length:len(speech)], is_final=True)
    else:
        results = speech2text(speech, is_final=True)
    nbests = [text for text, token, token_int, hyp in results]
    progress_output(nbests[0])

In [33]:
i = 10000
speech = wav.astype(np.float16)/32767.0 #32767 is the upper limit of 16-bit binary numbers and is used for the normalization of int to float.
sim_chunk_length = 640
results = speech2text2(speech=speech[i*sim_chunk_length:(i+1)*sim_chunk_length], is_final=False)

In [41]:
text, token, token_int, hyp_object = results_list[-1][0]

In [42]:
text

'あのーすいませんフランス料理屋を都内で探しているんですけどーえっとーそうだなちょっと家族の誕生日用に使うんですけどーえっとーえっとーあの近くかなにかいいお店ありますかね'

In [ ]:
8-16-16

In [13]:
import os
import json
import torch
import argparse
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from dotmap import DotMap
from tqdm import tqdm
import sys

sys.path.append('../..')
from src.datasets.dataset_timing_char import get_dataloader, get_dataset
from src.utils.utils import load_config
#from src.models.model import TimingEstimator

In [14]:
config_path = '../../configs/timing/test.json'
config = load_config(config_path)
# if args.gpuid >= 0:
config.gpu_device = 1

if config.cuda:
    device = torch.device('cuda:{}'.format(config.gpu_device))
else:
    device = torch.device('cpu')

In [16]:
#train_dataset = get_dataset(config, 'train')
val_dataset = get_dataset(config, 'valid', ['M1'])
#test_dataset = get_dataset(config, 'test')

#train_loader = get_dataloader(train_dataset, config, 'train')
val_loader = get_dataloader(val_dataset, config, 'valid')
# test_loader = get_loader(test_dataset, config, 'test')

 17%|█▋        | 1/6 [00:00<00:01,  3.11it/s]

525 503 523
111 89 109
215 197 217
440 423 443
104 82 102
160 138 158
276 254 274
274 252 272
129 107 127
68 46 66


 33%|███▎      | 2/6 [00:00<00:01,  3.59it/s]

307 285 305
124 102 122
87 70 90
158 136 156
35 31 51
46 44 64
100 95 115
117 112 132
64 42 62
114 104 124
288 266 286
71 65 85
49 32 52


 50%|█████     | 3/6 [00:00<00:00,  3.90it/s]

...

213 191 211
56 44 64
177 165 185
426 404 424
320 298 318
169 147 167
184 182 202
224 202 222
114 92 112


 83%|████████▎ | 5/6 [00:01<00:00,  2.97it/s]

505 503 523
145 123 143
103 81 101
99 77 97
101 99 119
112 110 130
60 38 58
64 42 62
58 40 60
150 129 149
83 80 100
63 41 61
132 110 130
269 247 267
212 190 210
213 191 211
163 145 165
168 146 166
101 98 118
109 92 112
74 55 75
98 89 109
58 55 75
96 74 94
64 42 62
71 69 89
374 352 372
38 33 53
94 74 94
176 154 174


100%|██████████| 6/6 [00:02<00:00,  2.92it/s]

83 61 81
109 87 107
118 116 136
145 123 143
106 84 104
47 42 62


In [20]:
cnt = 0
for batch in val_loader:
    if cnt >= 0:
        break
    cnt += 1
        
# chs = batch[0]
text = batch[1]
# turn = batch[2].to(device)
# #last_ipu = batch[3].to(self.device)
# targets = batch[4].to(device)
# feats = batch[5].to(device)
# input_lengths = batch[6] #.to(self.device)
target = batch[7]
# wav_lengths = batch[8] #.to(self.device)
wav_paths = batch[-1] #.to(self.device)

In [21]:
wav_paths

['/mnt/aoni04/jsakuma/data/ATR-Fujie/samples/wav/20131101-2_04/20131101-2_04_001_ch0.wav']

In [35]:
len(target[0])

380

In [25]:
text[0][-1]

'あとーすいませんえっと今夜ーと一人でえっとーどこは食べるところ探しているんですけどーえとー今どうしゃぶしていてーんでーこことをちょってーですけどーはいてーこのあるんでーこのーとを希望していでーとを希望していてーと思ってるんですけどーはなるんでーですけれどーでーでー'

In [11]:
if vad[i][:L].numpy().sum()>0:
    idx = np.where(vad[i][:L].numpy()==1)[0][-1]
else:
    idx = len(vad[i][:L].numpy())
    
idx

TypeError: only integer tensors of a single element can be converted to an index

In [22]:
# import time

# i=0

# start = time.perf_counter()

# wav = wavs[i][:wav_lengths[i]].detach().cpu().numpy()
# recognize(wav)

# end = time.perf_counter()
# print(end-start)

In [23]:
# import IPython
# rate = 16000
# IPython.display.Audio(wav, rate=rate)

In [24]:
# recognize_from_file(val_dataset[0][2])

In [25]:
def get_recognize_frame(data):
    output_list = []
    speech = data.astype(np.float16)/32767.0 #32767 is the upper limit of 16-bit binary numbers and is used for the normalization of int to float.
    sim_chunk_length = 800 # 640

    if sim_chunk_length > 0:
        for i in range(len(speech)//sim_chunk_length):
            results = speech2text(speech=speech[i*sim_chunk_length:(i+1)*sim_chunk_length], is_final=False)
            if results is not None and len(results) > 0:
                nbests = [text for text, token, token_int, hyp in results]
                text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
                #progress_output(nbests[0])
                output_list.append(text)
            else:
                #progress_output("")
                output_list.append("")

        results = speech2text(speech[(i+1)*sim_chunk_length:len(speech)], is_final=True)
    else:
        results = speech2text(speech, is_final=True)
    
    nbests = [text for text, token, token_int, hyp in results]
    if results is not None and len(results) > 0:
                nbests = [text for text, token, token_int, hyp in results]
                text = nbests[0] if nbests is not None and len(nbests) > 0 else ""
                #progress_output(nbests[0])
                output_list.append(text)
    else:
        #progress_output("")
        output_list.append("")
    return output_list
    #progress_output(nbests[0])
    

In [26]:
import time

wav = wavs[i][:wav_lengths[i]].detach().cpu().numpy()
start = time.perf_counter()
output = get_recognize_frame(wav)
end = time.perf_counter()
print(end-start, len(output))

22.090909585356712 430


In [27]:
import IPython
rate = 16000
IPython.display.Audio(wav, rate=rate)

In [40]:
wav = wavs[i][:wav_lengths[i]].detach().cpu().numpy()
start = time.perf_counter()

if vad[i][:L].numpy().sum()>0:
    idx = np.where(vad[i][:L].numpy()==1)[0][-1]
    N = len(vad[i][:L].numpy())-idx
else:
    idx = len(vad[i][:L].numpy())
    N = 0
    
    
wav = wav[:(idx+1)*800]
output2 = get_recognize_frame(wav)
tmp = [output2[-1]]*N
output2 += tmp
end = time.perf_counter()
print(end-start, len(output2))

6.743575740605593 223


In [37]:
output2[-1], output2[-2], output[-1]

('えっとランチで行きたいんですがーんーとできればそんなに高くなくてーんえーとランチメニューがお店以内がいいです',
 'えっとランチで行きたいんですがーんーとできればそんなに高くなくてーんえーとランチメニューがお店以',
 'えっとランチで行きたいんですがーんーとできればそんなに高くなくてーんえーとランチメニューがお店以内がいいです')

In [41]:
a = np.zeros(10)
a[:12]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [17]:
len(output), len(wav.astype(np.float16)/32767.0)

(430, 343200)

In [18]:
343200//(50*16)

429

In [16]:
output

['',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 '',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーと店探していただ',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーとお店探していただきたいんですけれど',
 'えーと店探していただきたいんですけれども',
 'えーと店探していただきたいんですけれども',
 'えーと店探していただきたいんですけれども',
 'えーと店探していただきたいんですけれども',
 'えーと店探していただきたいんですけれども',
 'えーと店探していただきたいんですけれども',
 'えーと店探していただきたいんですけれども',
 'えーと店探していただき

In [49]:
df = pd.DataFrame({'asr_recog': output})
df.tail()

,asr_recog
425,えーとお店探していただきたいんですけれどもーえーとですねあのー二人ーなんですけれどもえーとち...
426,えーとお店探していただきたいんですけれどもーえーとですねあのー二人ーなんですけれどもえーとち...
427,えーとお店探していただきたいんですけれどもーえーとですねあのー二人ーなんですけれどもえーとち...
428,えーとお店探していただきたいんですけれどもーえーとですねあのー二人ーなんですけれどもえーとち...
429,えーとお店探していただきたいんですけれどもーえーとですねあのー二人ーなんですけれどもえーとち...
